In [1]:
import tarfile

import tarfile
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
from sklearn.decomposition import NMF

import re
import math

from sklearn.decomposition import NMF

## Load Day 0 data 

In [2]:
dataset10 = pq.ParquetDataset("bigd/M148/hitdata7days/visitday=10/")

In [3]:
columns = ["cookieid","firsthitpagename","visitpagenum","ordernumber"]
data10 = dataset10.read(columns=columns).combine_chunks()

In [4]:
data_10 = dataset10.read().combine_chunks()

In [5]:
data10_pd = data10.to_pandas()

In [6]:
data10_pd.head(10)

,cookieid,firsthitpagename,visitpagenum,ordernumber
0,18976392225474675141320040725069689144,my account:sign in,168,None
1,22458651700155932098567309246136578371,search:results,2,None
2,81302138040689021608603450882312197152,None,143,None
3,59091259204920318791245315839306482106,index,55,None
4,53802118384124243215211591740223846690,index,34,None
5,78927590306810709598935516385592759617,my account:sign in,196,None
6,79419827933071974896753648306007745918,index,43,None
7,47316709324700216004627670073521110994,index,67,None
8,58602386511365160255081036280574911942,pdp:Nostalgia 12-Cup Hot Air Popcorn Maker - Red,83,None
9,73007192487732837592201854421032220814,index,15,None


In [7]:
data10_pd.shape

(6458609, 4)

## Load data for all 7 days 

In [18]:
dataset_all = pq.ParquetDataset("bigd/M148/hitdata7days")

In [19]:
columns = ["cookieid","firsthitpagename","visitpagenum","ordernumber"]
data_all = dataset_all.read(columns=columns).combine_chunks()

In [20]:
data_all_pd = data_all.to_pandas()

In [21]:
data_all_pd.tail(10)

,cookieid,firsthitpagename,visitpagenum,ordernumber,visitday
42730139,66766296197672086254571863561089794102,search:results,27,None,16
42730140,42336954984504414921774913107250050258,None,24,None,16
42730141,17693983337648543312395385556212515037,index,38,None,16
42730142,85153173066658247329001202340424923203,index,10,None,16
42730143,81128382723058624884928245544453957460,None,202,None,16
42730144,83103452230070488727514707142695403239,None,128,None,16
42730145,78978313603212467237685845152760013485,my account:sign in,15,None,16
42730146,38462509364520503943541087382094206104,my account:sign in,86,None,16
42730147,53686880511144008613364683438838426866,None,129,None,16
42730148,15214929409965487617343822276678408192,None,5,None,16


In [22]:
data_all_pd.shape

(42730149, 5)

## Subset data to get only users who ordered something

In [23]:
df_orders = data_all_pd[data_all_pd.ordernumber.notnull()]

In [24]:
## Fix the index
df_orders_index = df_orders.reset_index()
df_orders_index = df_orders_index.drop('index', axis=1)

In [25]:
cookieid_unique = df_orders_index["cookieid"].unique()

In [26]:
df_features = pd.DataFrame(0, index =range(0,len(cookieid_unique)), 
                               columns=["cookieid", "firsthitpagename", "mean(visitpagenum)"])

In [27]:
for i in range(0,len(cookieid_unique)):
    df_features.loc[i,"cookieid"] = cookieid_unique[i]
    
    df_features.loc[i,"mean(visitpagenum)"] = np.mean(df_orders[df_orders["cookieid"] == cookieid_unique[i]].visitpagenum)
    
    temp_string = df_orders_index[df_orders_index["cookieid"] == cookieid_unique[i]].firsthitpagename.head(1).item()
    
    if temp_string is None: 
        df_features.loc[i,"firsthitpagename"] = None 
    
    elif bool(re.search("index", temp_string)):
        df_features.loc[i,"firsthitpagename"] = "index"
        
    elif bool(re.search("my account", temp_string)):
        df_features.loc[i,"firsthitpagename"] = "my account"
        
    elif bool(re.search("pdp", temp_string)):
        df_features.loc[i,"firsthitpagename"] = "pdp"
    
    elif bool(re.search("cart", temp_string)):
        df_features.loc[i,"firsthitpagename"] = "cart"
    
    elif bool(re.search("search", temp_string)):
        df_features.loc[i,"firsthitpagename"] = "search"
        
    elif bool(re.search("checkout", temp_string)):
        df_features.loc[i,"firsthitpagename"] = "checkout"
        
    else:
        df_features.loc[i,"firsthitpagename"] = "other"  
    

In [28]:
df_features.shape

(103138, 3)

In [29]:
df_features.head(15)

,cookieid,firsthitpagename,mean(visitpagenum)
0,52882151623272742032948034078925051832,my account,27.0
1,79755164678756084974501978337153484560,index,125.0
2,5443955122341888424003495602170301473,index,88.5
3,15567279697543469666445773480724187157,my account,291.0
4,71667249622638493801005527041676393041,my account,31.0
5,61042077885887970553598772945168024836,index,62.0
6,50173036520586681086289374017581049714,index,122.0
7,75806096112973408034688549910937655265,index,42.0
8,69355983623503946334291902547906312305,pdp,62.5
9,32066281582833015954444801797185030134,my account,143.0


In [30]:
#df_features.to_csv('Marieth_features_all.csv')